In [149]:
import pickle
import os

import uuid

import pandas as pd

import mlflow

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline

In [168]:
year = 2021
month = 3 
taxi_type = "green"

input_file = f'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_{year:04d}-{month:02d}.parquet'

output_file = f"output/green/{taxi_type}{year:04d}-{month:02d}.parquet"

In [169]:
output_file

'output/green/green2021-03.parquet'

In [170]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("green-taxi-duration")

<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1733828272132, experiment_id='1', last_update_time=1733828272132, lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [171]:
#MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'
run_id = '55125dffde68494692293605dc53fd29'

logged_model = f'/workspaces/mlops-zoomcamp/04-orcherstration/web-service-mlflow/mlartifacts/1/{RUN_ID}/artifacts/model'
model= mlflow.pyfunc.load_model(logged_model)

In [172]:
model

mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.sklearn
  run_id: 55125dffde68494692293605dc53fd29

In [173]:
def generate_uuids(n):
    ride_ids = []
    for i in range(n):
        ride_ids.append(str(uuid.uuid4()))
    return ride_ids


def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    df['ride_id'] = generate_uuids(len(df))

    return df


def prepare_dictionaries(df: pd.DataFrame):
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']

    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts


def load_model(run_id):
    logged_model = f's3://mlflow-models-alexey/1/{run_id}/artifacts/model'
    model = mlflow.pyfunc.load_model(logged_model)
    return model


def save_results(df, y_pred, run_id, output_file):
    df_result = pd.DataFrame()
    df_result['ride_id'] = df['ride_id']
    df_result['lpep_pickup_datetime'] = df['lpep_pickup_datetime']
    df_result['PULocationID'] = df['PULocationID']
    df_result['DOLocationID'] = df['DOLocationID']
    df_result['actual_duration'] = df['duration']
    df_result['predicted_duration'] = y_pred
    df_result['diff'] = df_result['actual_duration'] - df_result['predicted_duration']
    df_result['model_version'] = run_id

    df_result.to_parquet(output_file, index=False)

In [174]:
prepare_dictionaries(df)

[{'PU_DO': '83_129', 'trip_distance': 1.56},
 {'PU_DO': '243_235', 'trip_distance': 0.96},
 {'PU_DO': '75_242', 'trip_distance': 9.93},
 {'PU_DO': '242_208', 'trip_distance': 2.57},
 {'PU_DO': '41_151', 'trip_distance': 0.8},
 {'PU_DO': '173_56', 'trip_distance': 1.12},
 {'PU_DO': '75_75', 'trip_distance': 0.38},
 {'PU_DO': '75_75', 'trip_distance': 0.38},
 {'PU_DO': '43_24', 'trip_distance': 1.73},
 {'PU_DO': '107_170', 'trip_distance': 0.0},
 {'PU_DO': '21_252', 'trip_distance': 26.72},
 {'PU_DO': '82_130', 'trip_distance': 0.0},
 {'PU_DO': '42_74', 'trip_distance': 1.3},
 {'PU_DO': '74_238', 'trip_distance': 3.08},
 {'PU_DO': '179_22', 'trip_distance': 17.03},
 {'PU_DO': '74_7', 'trip_distance': 3.74},
 {'PU_DO': '92_192', 'trip_distance': 1.25},
 {'PU_DO': '41_168', 'trip_distance': 3.0},
 {'PU_DO': '127_243', 'trip_distance': 1.23},
 {'PU_DO': '130_258', 'trip_distance': 2.7},
 {'PU_DO': '247_119', 'trip_distance': 1.09},
 {'PU_DO': '252_21', 'trip_distance': 25.81},
 {'PU_DO': '7

In [175]:
def apply_model(input_file, run_id, output_file):

    df = read_dataframe(input_file)    
    dicts = prepare_dictionaries(df)

    model = load_model(run_id)
    y_pred= model.predict(dicts)
    
    df_result["ride_id"] = df["ride_id"]
    df_result["lpep_pickup_datetime"] = df["lpep_pickup_datetime"]
    df_result["PULocationID"] = df["PULocationID"]
    df_result["DOLocationID"] = df["DOLocationID"]
    df_result["actual_duration"] = df["duration"]
    df_result["predicted_duration"] = y_pred
    df_result["diff"] = df_result["actual_duration"] - df_result["predicted_duration"]
    df_result['model_version'] = run_id

    df_result.to_parquet(output_file, index= False)


In [177]:
apply_model(input_file= input_file,run_id=run_id, output_file=output_file)

NoCredentialsError: Unable to locate credentials

In [ ]:
!ls output/green